# Install all prerequisities

In [ ]:
!sudo apt install build-essential -y;

In [ ]:
!sudo apt install iverilog verilator gtkwave -y;

In [ ]:
!pip3 install jinja2 pyverilog veriloggen numpy onnx

In [ ]:
!pip3 install torch torchvision

In [ ]:
!pip3 install pytest pytest-pythonpath

In [ ]:
!git clone https://github.com/knbi-pw/nngen.git

In [ ]:
!cd nngen && git checkout bugfix_onnx_reshape && cd ..

In [ ]:
!cd nngen && ls -la && python3 setup.py install && cd ../

# Restart the kernel to get nngen running in colab

In [ ]:
get_ipython().kernel.do_shutdown(True)

# NN stuff below

## Imports

In [1]:
from __future__ import absolute_import
from __future__ import print_function

import os
import sys
import math
import numpy as np
import PIL
import json

import torch
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd
import onnx

import nngen as ng

from veriloggen import *
import veriloggen.thread as vthread
import veriloggen.types.axi as axi

## ONNX tests

## Load ONNX

In [2]:
act_dtype=ng.int8
weight_dtype=ng.int8
bias_dtype=ng.int32
scale_dtype=ng.int8
with_batchnorm=True
disable_fusion=False

onnx_filename='/content/cnn_onnx_20220624_160819.onnx'

sim_filename=None
simtype=None


act_shape = (28, 28, 1)  # input image shape


In [3]:
# ONNX to NNgen
dtypes = {}
(outputs, placeholders, variables,
 constants, operators) = ng.from_onnx(onnx_filename,
                                      value_dtypes=dtypes,
                                      default_placeholder_dtype=act_dtype,
                                      default_variable_dtype=weight_dtype,
                                      default_constant_dtype=weight_dtype,
                                      default_operator_dtype=act_dtype,
                                      default_scale_dtype=scale_dtype,
                                      default_bias_dtype=bias_dtype,
                                      disable_fusion=disable_fusion)

## Quantize

In [6]:
# --------------------
# (2) Assign quantized weights to the NNgen operators
# --------------------
mean = 0.258
std = 0.437

if act_dtype.width > 8:
    act_scale_factor = 128
else:
    act_scale_factor = int(round(2 ** (act_dtype.width - 1) * 0.5))

input_name = 'act'

input_scale_factors = {input_name: act_scale_factor}
input_means = {input_name: mean * act_scale_factor}
input_stds = {input_name: std * act_scale_factor}

ng.quantize(outputs, input_scale_factors, input_means, input_stds)

## Assign hardware attributes

In [7]:
conv2d_par_ich=1
conv2d_par_och=1
conv2d_par_col=1
conv2d_par_row=1
conv2d_concur_och=None
conv2d_stationary='filter'
pool_par=1
elem_par=1

for op in operators.values():
    if isinstance(op, ng.conv2d):
        op.attribute(par_ich=conv2d_par_ich,
                     par_och=conv2d_par_och,
                     par_col=conv2d_par_col,
                     par_row=conv2d_par_row,
                     concur_och=conv2d_concur_och,
                     stationary=conv2d_stationary)

    if isinstance(op, (ng.avg_pool, ng.max_pool,
                       ng.avg_pool_serial, ng.max_pool_serial)):
        op.attribute(par=pool_par)

    if ng.is_elementwise_operator(op):
        op.attribute(par=elem_par)

## Generate IP

In [ ]:
outputs

In [ ]:

out = outputs['out']
silent=False
axi_datawidth=32
targ = ng.to_ipxact([out], 'lenet', silent=silent,
                    config={'maxi_datawidth': axi_datawidth})

## Save weights

In [12]:
weight_filename = "lenet_weights.npz"
chunk_size=64

param_data = ng.export_ndarray([out], chunk_size)
param_bytes = len(param_data)
np.savez_compressed(weight_filename, param_data)

## Zip IP

In [ ]:
!zip -r lenet.zip lenet_v1_0